In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from scipy import signal

import sys
import os
from os import getcwd
from os.path import dirname,abspath
import pathlib as pl


import tensorflow as tf
import keras as kr

import wfdb 
import neurokit2 as neuro



home_path = "C:\\sebin\\lab\\ecg"
git_path = home_path+"\\ecg_data_wfdb"

data_path = home_path+"\\"+"data"
file_name = None

output_path = home_path+"\\save\\china"

sys.path.append(git_path)
import use.mitbih_ecg.form as wf_ecg
from use.segment.pick import random_ecg
from use.segment.open import ecgtodf
from use.tensor.input import x_tensor_input,y_tensor_input



In [2]:
#china data : http://www.icbeb.org/CPSC2021
#annotation : https://archive.physionet.org/physiobank/annotations.shtml

In [42]:
def china_ecg_resampling(path):
    #path is .atr제거한 형식이다
    #null값은 -100으로 표시 


    #open data
    data = wfdb.rdsamp(path)
    ecg = data[0]
    reference = data[1]
    ann = wfdb.rdann(path,extension="atr")


    #default ecg signal(sampling rate: 200hz/s)
    ecg=ecg.T
    ecg = ecg[1]
    n = len(ecg)
    
    ann_x = ann.sample
    ann_c = ann.symbol
    if ann_x[0]<200:
        ann_x = ann_x[1:]
        ann_c = ann_c[1:]
    ann_y = np.full((n),-100,dtype='float64')
    for i in ann_x:
        #print(i)
        ann_y[i-1] = ecg[i]

    
    original = {'ecg':ecg,'ann_x':ann_x,'ann_y':ann_y,'ann_c':ann_c}


    
    
    re_ecg = signal.resample_poly(ecg,9,5)

    n = len(re_ecg)

    re_ann_x = neuro.ecg_findpeaks(re_ecg,360)['ECG_R_Peaks']
    re_ann_y = np.full((n),-100,dtype='float64')

    for i in re_ann_x:
        #print(i)
        re_ann_y[i] = re_ecg[i]
    
    
    sampling = {'ecg':re_ecg,'ann_x':re_ann_x,'ann_y':re_ann_y,'ann_c':ann_c}

    if len(original['ann_c'])!=len(sampling['ann_c']):
        raise Exception("original ecg and sampling ecg segment number is not equall")
        
    return original,sampling

In [104]:
def segementation(file_name,input_folder,output_folder):
    print(file_name+" start")
    default,sampling = china_ecg_resampling(input_folder+"\\"+file_name)

    cnt = 0
    for i in range(len(default['ann_c'])):
        if default['ann_c'][i]!= "N":
            cnt+=1
    if cnt==0:
       return -1
    
    seg = []
    ann = []
    size = sampling['ann_x'].size
    for i in range(size):
        if sampling['ann_c'][i]!='N':
            r = sampling['ann_x'][i]

            sepfrom = r-144
            septo = r+144

            if sepfrom <= 0:
                continue
            if septo >= sampling['ann_x'][-1]:
                break

            seg.append(sampling['ecg'][sepfrom:septo])
            ann.append(sampling['ann_c'][i])

    if seg != []:
        df = pd.DataFrame(seg)
        folder = output_folder+"\\"+file_name
        if not os.path.exists(folder):
            os.makedirs(folder)
            for i in range(len(df.index)):
                ecg_data = pd.DataFrame(df.iloc[i])
                ann_data = ann[i]

                print(ecg_data)
                print(ann_data)
                p = folder + "\\" + str(ann_data) + "_" + str(i) + ".csv"
                ecg_data.to_csv(p,header=False,index=False)
        print(file_name+" end")

In [105]:
input_folder = abspath(home_path+"\data\china\\training_1")
output_folder = abspath(home_path+"\save\china")

In [107]:
data_file = []
for i in os.listdir(input_folder):
    if i.endswith(".dat"):
        data_file.append(i.split(".dat")[0])

In [108]:
for name in data_file:
    segementation(name,input_folder,output_folder)

data_0_1 start
            0
0    0.221888
1    0.231549
2    0.235740
3    0.236483
4    0.237295
..        ...
283 -0.148896
284 -0.144547
285 -0.140148
286 -0.137980
287 -0.137558

[288 rows x 1 columns]
V
data_0_1 end
data_0_10 start
data_0_11 start
data_0_12 start
data_0_13 start
data_0_14 start
data_0_15 start
data_0_2 start
data_0_3 start
data_0_4 start
data_0_5 start
data_0_6 start
data_0_7 start
data_0_8 start
data_0_9 start
data_10_1 start
data_10_10 start
            0
0    4.673195
1    4.692988
2    4.714945
3    4.703794
4    4.695118
..        ...
283  4.416917
284  4.391483
285  4.370231
286  4.377241
287  4.373157

[288 rows x 1 columns]
V
            1
0    4.928422
1    4.900874
2    4.853315
3    4.805023
4    4.791765
..        ...
283  3.931252
284  3.946992
285  3.949373
286  3.954772
287  3.973374

[288 rows x 1 columns]
V
            2
0    4.855673
1    4.842692
2    4.817273
3    4.794555
4    4.797780
..        ...
283  4.877526
284  4.846478
285  4.838266
2

IndexError: list index out of range